# Scraping Concerts - Lab

## Introduction

Now that you've seen how to scrape a simple website, it's time to again practice those skills on a full-fledged site!
In this lab, you'll practice your scraping skills on a music website: https://www.residentadvisor.net.
## Objectives

You will be able to:
* Scrape events from a website
* Follow links to those events to retrieve further information
* Clean and store scraped data

## View the Website

For this lab, you'll be scraping the https://www.residentadvisor.net website. Start by navigating to the events page [here](https://www.residentadvisor.net/events) in your browser.

<img src="images/ra.png">

In [ ]:
#Load the https://www.residentadvisor.net/events page in your browser.

## Open the Inspect Element Feature

Next, open the inspect element feature from your web browser in order to preview the underlying HTML associated with the page.

In [ ]:
#Open the inspect element feature in your browser

## Write a Function to Scrape all of the Events on the Given Page Events Page

The function should return a Pandas DataFrame with columns for the Event_Name, Venue, Event_Date and Number_of_Attendees.

In [1]:
import re
import numpy as np
import pandas as pd
import requests
from bs4 import BeautifulSoup
import time

In [21]:
response = requests.get('https://www.residentadvisor.net/events/us/texas')
soup = BeautifulSoup(response.content, 'html.parser')

In [22]:
event_listings = soup.find('div', id = 'event-listing')

In [23]:
entries = event_listings.findAll('li')
print(len(entries), entries[0])

24 <li><p class="eventDate date"><a href="/events.aspx?ai=318&amp;v=day&amp;mn=10&amp;yr=2019&amp;dy=8"><span>Tue, 08 Oct 2019 /</span></a></p></li>


In [24]:
rows = []
for entry in entries:
    date = entry.find('p', class_="eventDate date")
    event = entry.find('h1', class_="event-title")
    if event:
        details = event.text.split(' at ')
        event_name = details[0].strip()
        venue = details[1].strip()
        try:
            n_attendees = int(re.match("(\d*)", entry.find('p', class_="attending").text)[0])
        except:
            n_attendees = np.nan
        rows.append([event_name, venue, cur_date, n_attendees])
    elif date:
        cur_date = date.text
    else:
        continue
df = pd.DataFrame(rows)
df.columns = ['Event_Name', 'Venue', 'Event_Date', 'Number_of_Attendees']
df.head()

,Event_Name,Venue,Event_Date,Number_of_Attendees
0,"Underground Tuesdays feat. Iwonder, Lowtide, D...","Empire Control Room & Garage, Austin","Tue, 08 Oct 2019 /",NaN
1,Ayokay 10/9,"The Parish, Austin","Wed, 09 Oct 2019 /",3.0
2,Steady Rhythm 8 Year Anniversary,"Plush ATX, Austin","Wed, 09 Oct 2019 /",2.0
3,Flamingosis + Boombaptist Official 2019 ACL Fe...,"The Parish, Austin","Thu, 10 Oct 2019 /",4.0
4,Kinda Super Disco - Hot Since 82,"Ayva Center, Houston","Fri, 11 Oct 2019 /",14.0


In [25]:
def scrape_events(events_page_url):
    response = requests.get(events_page_url)
    soup = BeautifulSoup(response.content, 'html.parser')
    event_listings = soup.find('div', id = 'event-listing')
    entries = event_listings.findAll('li')
    rows = []
    for entry in entries:
        #Is it a date? If so, set current date.
        date = entry.find('p', class_="eventDate date")
        event = entry.find('h1', class_="event-title")
        if event:
            details = event.text.split(' at ')
            event_name = details[0].strip()
            venue = details[1].strip()
            try:
                n_attendees = int(re.match("(\d*)", entry.find('p', class_="attending").text)[0])
            except:
                n_attendees = np.nan
            rows.append([event_name, venue, cur_date, n_attendees])
        elif date:
            cur_date = date.text
        else:
            continue
    df = pd.DataFrame(rows)
    df.columns = ["Event_Name", "Venue", "Event_Date", "Number_of_Attendees"]
    return df

In [33]:
df = scrape_events('https://www.residentadvisor.net/events/us/texas')
df.head()

,Event_Name,Venue,Event_Date,Number_of_Attendees
0,"Underground Tuesdays feat. Iwonder, Lowtide, D...","Empire Control Room & Garage, Austin","Tue, 08 Oct 2019 /",NaN
1,Ayokay 10/9,"The Parish, Austin","Wed, 09 Oct 2019 /",3.0
2,Steady Rhythm 8 Year Anniversary,"Plush ATX, Austin","Wed, 09 Oct 2019 /",2.0
3,Flamingosis + Boombaptist Official 2019 ACL Fe...,"The Parish, Austin","Thu, 10 Oct 2019 /",4.0
4,Kinda Super Disco - Hot Since 82,"Ayva Center, Houston","Fri, 11 Oct 2019 /",14.0


## Write a Function to Retrieve the URL for the Next Page

In [26]:
soup.find('a', attrs = {'ga-event-action': 'Next '}).attrs['href']

'/events/us/texas/week/2019-10-14'

In [27]:
def next_page(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    url_ext = soup.find('a', attrs={'ga-event-action':"Next "}).attrs['href']
    next_page_url = "https://www.residentadvisor.net" + url_ext
    return next_page_url

In [31]:
next_page('https://www.residentadvisor.net/events/us/texas')

'https://www.residentadvisor.net/events/us/texas/week/2019-10-14'

## Scrape the Next 1000 Events for Your Area

Display the data sorted by the number of attendees. If there is a tie for the number attending, sort by event date.

In [41]:
dfs = []
total_rows = 0
cur_url = "https://www.residentadvisor.net/events/us/texas"
while total_rows <= 63:
    df = scrape_events(cur_url)
    dfs.append(df)
    total_rows += len(df)
    cur_url = next_page(cur_url)
    time.sleep(.2)
df = pd.concat(dfs)
df = df.iloc[:1000]
print(len(df))
df.head()

65


,Event_Name,Venue,Event_Date,Number_of_Attendees
0,"Underground Tuesdays feat. Iwonder, Lowtide, D...","Empire Control Room & Garage, Austin","Tue, 08 Oct 2019 /",NaN
1,Ayokay 10/9,"The Parish, Austin","Wed, 09 Oct 2019 /",3.0
2,Steady Rhythm 8 Year Anniversary,"Plush ATX, Austin","Wed, 09 Oct 2019 /",2.0
3,Flamingosis + Boombaptist Official 2019 ACL Fe...,"The Parish, Austin","Thu, 10 Oct 2019 /",4.0
4,Kinda Super Disco - Hot Since 82,"Ayva Center, Houston","Fri, 11 Oct 2019 /",14.0


### Note:
Texas' page does not go beyond the 65th event currently, I assume due to Christmas weeks being empty.

## Summary 

Congratulations! In this lab, you successfully scraped a website for concert event information!